In [1]:
import os 
from pathlib import Path 

import numpy as np
import pandas as pd

import xarray as xr
import netCDF4 as nc
import wrf

import matplotlib.pyplot as plt

In [2]:
# Define directories
root_dir = Path().resolve().parents[0]
data_dir = root_dir / 'data' / 'geospatial'
wrf_fpath = data_dir / 'wrf-vertical-profile' / 'wrfout-2008-08-vertprofile.nc'
clc_fpath = data_dir / 'clemesha-clc' / 'cldalb-sci.nc'

# Open datasets
clc_ds = xr.open_dataset(clc_fpath)
wrf_ds = xr.open_dataset(wrf_fpath)

In [3]:
# Make CLC Binary
cldalb = clc_ds['cldalb']
cldalb_binary = xr.where(cldalb > 8.5, 1, 0)

In [4]:
# Get WRF CLC
# define cloud levels
low_cloud=120 
mid_cloud=2000
high_cloud=5000

wrf_ncfile = nc.Dataset(wrf_fpath)
cloudFrac=wrf.g_cloudfrac.get_cloudfrac(wrf_ncfile, timeidx=wrf.ALL_TIMES ,vert_type='height_msl',
                                        low_thresh=low_cloud,
                                        mid_thresh=mid_cloud,
                                        high_thresh=high_cloud)

# Get only low clouds
wrf_low_clc = cloudFrac.sel(low_mid_high='low')

In [5]:
# Format WRF data so we can join to GOES CLC data
wrf_low_clc_values = wrf_low_clc.values
wrf_lat = wrf_low_clc['XLAT'].values[:, 0]
wrf_lon = wrf_low_clc['XLONG'].values[0,:]
wrf_time = wrf_low_clc['Time'].values

wrf_low_clc_da = xr.DataArray(
    data=wrf_low_clc_values,
    dims=['time', 'lat', 'lon'],
    coords = {'time' : wrf_time, 'lat' : wrf_lat, 'lon' : wrf_lon}
)



In [9]:
# Save out dataarray
out_fpath = str(wrf_fpath).split('/')[:-1]
out_fpath = '/'.join(out_fpath) + '/wrf_low_clc.nc'
wrf_low_clc_da.to_netcdf(out_fpath)

# For memory reasons - START HERE 


In [ ]:
import os 
from pathlib import Path 

import numpy as np
import pandas as pd

import xarray as xr
import netCDF4 as nc
import wrf

import matplotlib.pyplot as plt

# Define directories
root_dir = Path().resolve().parents[0]
data_dir = root_dir / 'data' / 'geospatial'
wrf_fpath = data_dir / 'wrf-vertical-profile' / 'wrf_low_clc.nc'
clc_fpath = data_dir / 'clemesha-clc' / 'cldalb-sci.nc'

# Open datasetsa
clc_ds = xr.open_dataset(clc_fpath)
wrf_low_cloud = xr.open_dataarray(wrf_fpath)

# Make binary albedo
cldalb = clc_ds['cldalb']
cldalb = xr.where(cldalb > 8.5, 1, 0)

cldalb = cldalb.reindex_like(wrf_low_cloud, method='nearest')

ds = xr.Dataset({'cldalb_binary' : cldalb, 'wrf_low_cld' : wrf_low_cloud})
ds['wrf_binary'] = xr.where(wrf_low_cloud > 20, 1, 0)

In [15]:
ds

<xarray.Dataset> Size: 116MB
Dimensions:        (lon: 258, lat: 234, time: 96)
Coordinates:
  * lon            (lon) float32 1kB -121.3 -121.2 -121.2 ... -118.6 -118.6
  * lat            (lat) float32 936B 33.51 33.52 33.53 ... 35.49 35.49 35.5
  * time           (time) datetime64[ns] 768B 2008-08-17 ... 2008-08-25T23:00:00
Data variables:
    cldalb_binary  (time, lat, lon) int64 46MB 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1
    wrf_low_cld    (time, lat, lon) float32 23MB ...
    wrf_binary     (time, lat, lon) int64 46MB 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0